# **Introduction**

Medical costs are any expenses incurred for the prevention or treatment of disease or injury. Many things, such as prescriptions, doctor and hospital visits, can be counted among medical costs. Medical expenses are affected by many factors. In order to understand medical costs correctly, it is necessary to have detailed knowledge about all these factors. Although it is not ideal for such a study, only the most basic data of the patients are available in the data set. And the aim of this study is to analyze this data set, to reveal which factors affect health expenditures the most, and to predict health expenditures with the help of machine learning.

# **Data**

* **age** : Age of primary beneficiary
* **sex** : Insurance contractor gender, female, male
* **bmi** : Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
* **children**: Number of children covered by health insurance / Number of dependents
* **smoker**: Smoking
* **region**: The beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
* **charges**: Individual medical costs billed by health insurance

# **Libraries**

In [1]:
import numpy as np
import pandas as pd

import scipy
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE 

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# **Explore Data**

In [2]:
df_raw = pd.read_csv('../input/insurance/insurance.csv')
df_raw.head()

In [3]:
df_raw.info()

In [4]:
df_raw.describe(include="all")

# **Visual EDA**

In this section, the data set is analyzed visually. Preprocessing is not required at this stage, since the data set is very clean and has no missing data. We first start the visualization with the correlation between the features. As you can see, smoker, age, and bmi features are correlated with the charges feature. And in the scatter matrix below, it is clearly seen that age and bmi are related to charges.

In [5]:
df_corr = df_raw.copy()

df_corr["sex"] = df_corr["sex"].replace({'female': "0", 'male': "1"})
df_corr["smoker"] = df_corr["smoker"].replace({'no': "0", 'yes': "1"})
df_corr["region"] = df_corr["region"].replace({'southwest': "0", 'southeast': "1", 'northwest': "2", 'northeast': "3"})

LABELS = ["sex","smoker", "region"]

int_label = lambda x: x.astype('int64')
df_corr[LABELS] = df_corr[LABELS].apply(int_label, axis=0)

#mask = np.triu(np.ones_like(df_corr.corr()))
plt.figure(figsize = (12, 9))
s = sns.heatmap(df_corr.corr(),
               annot = True,
               #mask = mask,
               cmap = 'RdBu',
               vmin = -1,
               vmax = 1)

s.set_yticklabels(s.get_yticklabels(), rotation = 0, fontsize = 12)
s.set_xticklabels(s.get_xticklabels(), rotation = 90, fontsize = 12)
plt.title("Correlation Heatmap")
plt.show()

In [6]:
fig = px.scatter_matrix(df_raw,dimensions=["age", "bmi", "charges"],
                        color="smoker", symbol="smoker", opacity = 1,
                        color_discrete_sequence=["red", "blue"]
                       )

fig.update_layout(title="Scatter matrix",
                  legend_title = "Smoker",
                  title_x=0.5, title_y=0.95)
fig.show()

## **CHARGES**

Medical costs are usually concentrated between 2000 - 14000, with a few values going up to 60000. Therefore, charges have a right-skewed distribution.

In [7]:
n_data = len(df_raw["charges"]) 
n_bins = np.sqrt(n_data)
n_bins = int(n_bins)

fig = px.histogram(df_raw, x="charges", nbins=n_bins, marginal="box")
fig.update_layout(xaxis_title = 'Charges',
                  yaxis_title = 'Count',
                  title_text='Charges distribution', 
                  title_x=0.5, title_y=0.95)
fig.show()

## **SMOKING**

The difference between smokers and non-smokers is clearly visible.

**For non-smokers:**
* The charges  are concentrated around 3700
* The median value is 7345
* The mean value is 8434

**For smokers:**
* The charges  are concentrated around 20000 and 40000
* The median value is 34000
* The mean value is 32000

In [8]:
fig = go.Figure()


fig.add_trace(go.Violin(x=df_raw[df_raw["smoker"] == "no"]["smoker"], 
                        y=df_raw[df_raw["smoker"] == "no"]["charges"],
                        name="No",box_visible=True, meanline_visible=True, points="all"))

fig.add_trace(go.Violin(x=df_raw[df_raw["smoker"] == "yes"]["smoker"], 
                        y=df_raw[df_raw["smoker"] == "yes"]["charges"],
                        name="Yes",box_visible=True, meanline_visible=True, points="all"))

fig.update_layout(legend_title = 'Smoker',
                  xaxis_title = 'Smoker',
                  yaxis_title = 'Charges',
                  title_text='Smoking and Charges', 
                  title_x=0.5, title_y=0.9)
fig.show()

## **AGE**

As can be seen in the age distribution graph, there is a concentration in the 18-19 age range. The average age range is seen as 39-40. In addition, it is seen that the total number of men and women and their age distribution are similar to each other.

In [9]:
n_data = len(df_raw["age"]) 
n_bins = np.sqrt(n_data)
n_bins = int(n_bins)

fig = px.histogram(df_raw, x="age", color="sex", nbins=n_bins, marginal="box")

fig.update_layout(xaxis_title = 'Age',
                  yaxis_title = 'Count',
                  legend_title = 'Sex',
                  title_text='Age distribution', 
                  title_x=0.475, title_y=0.95)

fig.show()

We've seen the difference in medical costs between smokers and non-smokers in previous charts. Here, in addition, we see the effects of age and smoking together. When we look at the OLS trendline values, we see that the medical expenses of smokers increase more with age compared to non-smokers. It appears that smokers have an average of 38 dollars more medical costs per age than non-smokers.

**For smokers:**
* **OLS trendline:** 305.238 * age + 20294.1 

**For non-smokers:**
* **OLS trendline:** 267.249 * age - 2091.42

In [10]:
fig = px.scatter(df_raw, x="age", y="charges", facet_col="smoker", 
                 color="smoker", color_discrete_sequence=["red", "blue"], 
                 trendline='ols', hover_data=df_raw.columns)

fig.update_layout(legend_title = 'Smoker',
                  title_text='Age and Smoking', 
                  title_x=0.5, title_y=0.95)

fig.show()

In this section, we see in more detail the difference between smokers and non-smokers in certain age groups. This shows that there is a significant medical cost gap between smokers and non-smokers, regardless of young or old age.

**For 18-30 age group:**
* Median value of non-smokers 2755
* Median value of smokers 33475

**For 31-49 age group:**
* Median value of non-smokers 7149
* Median value of smokers 35491

**For 50-64 age group:**
* Median value of non-smokers 12029
* Median value of smokers 43696

In [11]:
bins = [17, 30, 49, 64]
labels = ["18-30","31-49","50-64"]
df_raw['age_bins'] = pd.cut(df_raw['age'], bins=bins, labels=labels)

fig = px.box(df_raw.sort_values(["age_bins", "smoker"]), x="age_bins", y="charges",
             color="smoker", points="all", hover_data=df_raw.columns)

fig.update_layout(xaxis_title = 'Age groups',
                  yaxis_title = 'Charges',
                  legend_title = 'Smoker',
                  title_text='Age and Smoking', 
                  title_x=0.5, title_y=0.95)
fig.show()

## **BMI**

In the previous sections, we saw that bmi and medical costs are correlated at a certain level. In this section, the effect of bmi on medical cost is analyzed in more detail. We started this analysis by visualizing the bmi distribution. As can be seen below, bmi has a symmetrical distribution and its median value is 30.4

In [12]:
n_data = len(df_raw["bmi"]) 
n_bins = np.sqrt(n_data)
n_bins = int(n_bins)

fig = px.histogram(df_raw, x="bmi", nbins=n_bins, marginal="box")

fig.update_layout(xaxis_title = 'BMI',
                  yaxis_title = 'Count',
                  title_text='BMI distribution', 
                  title_x=0.5, title_y=0.95)
fig.show()

We saw in the first section that there is a correlation between bmi and charges. Here, in addition, we see the effects of bmi and smoking together. When we look at the OLS trendline values, we see that the medical expenses of smokers increase more with bmi compared to non-smokers. It is seen that smokers have an average of 1390 dollars more medical cost per bmi than non-smokers.

**For smokers:**
* **OLS trendline:** 1473.11 * bmi + -13186.6

**For non-smokers:**
* **OLS trendline:** 83.3506 * bmi - 5879.42

In [13]:
fig = px.scatter(df_raw, x="bmi", y="charges", facet_col="smoker", 
                 color="smoker", color_discrete_sequence=["red", "blue"], 
                 trendline='ols', hover_data=df_raw.columns)

fig.update_layout(legend_title = 'Smoker',
                  title_text='BMI and Smoking', 
                  title_x=0.5, title_y=0.95)
fig.show()

As expected, there is a clear medical cost difference between smokers and non-smokers at all bmi levels. What is striking here is that the difference in medical expenses between smokers and non-smokers in the obesity group is much higher than in other groups.

**For underweight group:**
* Median value of non-smokers 3732
* Median value of smokers 15006

**For healthy weight group:**
* Median value of non-smokers 6593
* Median value of smokers 19479

**For overweight group:**
* Median value of non-smokers 7063
* Median value of smokers 21215

**For obesity group:**
* Median value of non-smokers 8076
* Median value of smokers 40904

In [14]:
bins = [15, 18.499, 24.999, 29.999, 54]
labels = ["Underweight","Healthy Weight","Overweight", "Obesity"]
df_raw['bmi_bins'] = pd.cut(df_raw['bmi'], bins=bins, labels=labels)

fig = px.box(df_raw.sort_values(["bmi_bins", "smoker"]), x="bmi_bins", y="charges",
             color="smoker", points="all", hover_data=df_raw.columns)

fig.update_layout(xaxis_title = 'BMI groups',
                  yaxis_title = 'Charges',
                  legend_title = 'Smoker',
                  title_text='BMI and Smoking', 
                  title_x=0.5, title_y=0.95)
fig.show()

In this section, we have compiled previous analyzes of the impact of smoking, bmi, and age on charges. The point to be noted here is that smokers' medical costs are divided into two groups. The reason for this is the joint effect of bmi and smoking. The medical cost values of those who smoke and are included in the obesity group are much higher than the others. In addition, the medical costs of a group of non-smokers are quite high compared to the other non-smokers. But we can't explain this with either bmi or smoking at the moment.

In [15]:
fig = px.scatter(df_raw.sort_values(["smoker","bmi"], ascending=False), 
                 x="age", y="charges", facet_col="smoker",color="bmi_bins",
                 color_discrete_sequence=["red", "blue","green","orange"],
                 hover_data=df_raw.columns)

fig.update_layout(legend_title = 'BMI',
                  title_text='Age - Smoking - BMI', 
                  title_x=0.47, title_y=0.95)
fig.show()

## **SEX**

In this section, the distribution of health expenditures for women and men is visualized.

**For women:**
* Median value is 9412
* Mean value is 12569

**For men:**
* Median value of non-smokers 9369
* Median value of smokers 13956

As can be seen, the median value for women is slightly higher than that for men, while the average value for men is considerably higher than for women.

In [16]:
fig = go.Figure()


fig.add_trace(go.Violin(x=df_raw[df_raw["sex"] == "female"]["sex"], 
                        y=df_raw[df_raw["sex"] == "female"]["charges"],
                        name="Female",box_visible=True, meanline_visible=True, points="all"))

fig.add_trace(go.Violin(x=df_raw[df_raw["sex"] == "male"]["sex"], 
                        y=df_raw[df_raw["sex"] == "male"]["charges"],
                        name="Male",box_visible=True, meanline_visible=True, points="all"))

fig.update_layout(legend_title = 'Sex',
                  xaxis_title = 'Sex',
                  yaxis_title = 'Charges',
                  title_text='Sex and Charges', 
                  title_x=0.48, title_y=0.9)
fig.show()

As can be seen in the two graphs below, the number of smokers and obesity in men is higher than in women. The main difference in the mean value in the chart above may be due to this.

**For women:**
* Number of smokers 115
* Number of obesity 334

**For men:**
* Number of smokers 159
* Number of obesity 373

In [17]:
fig = px.histogram(df_raw, x="sex", color="sex", pattern_shape="smoker", 
                   text_auto=True, pattern_shape_sequence=["x", ""])

fig.update_layout(legend_title = 'Sex, Smoker',
                  xaxis_title = 'Sex',
                  yaxis_title = 'Count',
                  title_text='Sex and Smoking', 
                  title_x=0.48, title_y=0.95)
fig.show()

In [18]:
fig = px.histogram(df_raw.sort_values(["sex","bmi_bins"]), x="sex",color="bmi_bins",
                   color_discrete_sequence =["orange", "green", "blue","red"], text_auto=True)

fig.update_layout(legend_title = 'BMI',
                  xaxis_title = 'Sex',
                  yaxis_title = 'Count',
                  title_text='Sex and BMI', 
                  title_x=0.46, title_y=0.95)
fig.show()

## **REGION**

In this section, we look at the distribution of health expenditures for regions. The main purpose here is to compare the medical costs of regions.

**For the northeast region:**
* Median value is 10057
* Mean value is 13406

**For the northwest region:**
* Median value is 8965
* Mean value is 12417

**For the southeast region:**
* Median value is 9294
* Mean value is 14735

**For the southwest region:**
* Median value is 8798
* Mean value is 12346

As can be seen, the northeast and southeast regions have the highest median and mean values.

In [19]:
fig = go.Figure()


fig.add_trace(go.Violin(x=df_raw[df_raw["region"] == "northeast"]["region"], 
                        y=df_raw[df_raw["region"] == "northeast"]["charges"],
                        name="northeast",box_visible=True, meanline_visible=True, points="all"))

fig.add_trace(go.Violin(x=df_raw[df_raw["region"] == "northwest"]["region"], 
                        y=df_raw[df_raw["region"] == "northwest"]["charges"],
                        name="northwest",box_visible=True, meanline_visible=True, points="all"))

fig.add_trace(go.Violin(x=df_raw[df_raw["region"] == "southeast"]["region"], 
                        y=df_raw[df_raw["region"] == "southeast"]["charges"],
                        name="southeast",box_visible=True, meanline_visible=True, points="all"))

fig.add_trace(go.Violin(x=df_raw[df_raw["region"] == "southwest"]["region"], 
                        y=df_raw[df_raw["region"] == "southwest"]["charges"],
                        name="southwest",box_visible=True, meanline_visible=True, points="all"))

fig.update_layout(legend_title = 'Region',
                  xaxis_title = 'Region',
                  yaxis_title = 'Charges',
                  title_text='Region and Charges', 
                  title_x=0.5, title_y=0.9)
fig.show()

As can be seen in the two graphs below, the number of smokers is higher in the southeast and northeast regions, respectively, than in other regions. The number of obesity in the southeast region is quite high compared to other regions, but the northeast region has the least number of obesity. Based on this, it can be said that the reason why the southeastern region has higher medical cost values may be the excess of the number of smokers and obese. And the reason for the high medical costs of the northeastern region may be the high number of smokers.

**For the northeast region:**
* Number of smokers 67
* Number of obesity 143

**For the northwest region:**
* Number of smokers 58
* Number of obesity 148

**For the southeast region:**
* Number of smokers 91
* Number of obesity 243

**For the southwest region:**
* Number of smokers 58
* Number of obesity 173


In [20]:
fig = px.histogram(df_raw.sort_values("region"), x="region", color="smoker", text_auto=True)

fig.update_layout(legend_title = 'Smoker',
                  xaxis_title = 'Region',
                  yaxis_title = 'Count',
                  title_text='Region and Smoking', 
                  title_x=0.48, title_y=0.95)
fig.show()

In [21]:
fig = px.histogram(df_raw.sort_values(["region","bmi_bins"]), x="region",color="bmi_bins",
                   color_discrete_sequence =["orange", "green", "blue","red"], text_auto=True)

fig.update_layout(legend_title = 'BMI',
                  xaxis_title = 'Region',
                  yaxis_title = 'Count',
                  title_text='Region and BMI', 
                  title_x=0.46, title_y=0.95)
fig.show()

# **Data preprocessing**

In this part, the data is prepared for machine learning models. Since there is no missing data and the data is quite clean, there is not much to be done. Categorical features such as sex, smoker, region are simply encoded and their data types are corrected. In addition, some features that will not be used in the model are dropped.

In [22]:
df_pre = df_raw.copy()

df_pre["sex"] = df_pre["sex"].replace({'female': "0", 'male': "1"})
df_pre["smoker"] = df_pre["smoker"].replace({'no': "0", 'yes': "1"})
df_pre["region"] = df_pre["region"].replace({'northeast': "0", 'northwest': "1",
                                                'southeast' : "2", 'southwest' : "3"})

LABELS = ["sex","smoker", "region"]
int_label = lambda x: x.astype('int64')
df_pre[LABELS] = df_pre[LABELS].apply(int_label, axis=0)
df_pre.drop(['age_bins', 'bmi_bins'], axis=1, inplace=True)

# **Train test split**

In this section, the data is divided into train and test set so that we can test machine learning models on unseen data.

In [23]:
X = df_pre.drop(columns=['charges']).copy()
y = df_pre['charges'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('X_train shape : {} - y_train shape : {}'.format(X_train.shape, y_train.shape))
print('X_test shape  : {}  - y_test shape  : {}'.format(X_test.shape, y_test.shape))

# **Random Forest Regressor**

In this section, medical cost prediction is made using machine learning models. After the data set is analyzed, it is seen that ensemble methods are more appropriate to make these predictions. Therefore, RandomForestRegressor and GradientBoostingRegressor models were chosen. Then, hyperparameters are searched using the GridSearchCV method to find the ideal model. In this step, hyperparameter scores were compared using the RMSE metric.

After selecting the appropriate hyperparameters, the models were tested on unseen data. When evaluated over R squared scores, it was observed that both models perform close to each other. A more detailed examination is done in the testing section.

In [24]:
param_grid = {
    'n_estimators': [500, 1000, 2000],
    'max_features': ['auto'],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 6, 8],
    'min_samples_leaf': [2, 6, 8]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator = rf, scoring = 'neg_root_mean_squared_error',
                           param_grid = param_grid, cv = 3)

grid_search.fit(X_train, y_train)

grid_cv_results_df = pd.DataFrame(grid_search.cv_results_)
columns = ['param_n_estimators', 'param_min_samples_split', 'param_min_samples_leaf', 'param_max_features', 
           'param_max_depth', 'mean_test_score','std_test_score', 'rank_test_score']

grid_cv_results_df[columns].sort_values("rank_test_score").head()

In [25]:
n_estimators = grid_search.best_params_["n_estimators"] 
min_samples_split = grid_search.best_params_["min_samples_split"] 
min_samples_leaf = grid_search.best_params_["min_samples_leaf"] 
max_features = grid_search.best_params_["max_features"] 
max_depth = grid_search.best_params_["max_depth"]

best_rf = RandomForestRegressor(n_estimators = n_estimators, min_samples_split = min_samples_split, 
                                min_samples_leaf = min_samples_leaf, max_features = max_features, 
                                max_depth = max_depth, random_state = 42)


best_rf.fit(X_train, y_train)

print('R-Squared score on training data : {}'.format(best_rf.score(X_train, y_train)))
print('R-Squared score on test data     : {}'.format(best_rf.score(X_test, y_test)))

# **Gradient Boosting Regressor**

In [26]:
param_grid = {
    'n_estimators': [40, 50],
    'learning_rate': [0.01,  0.1],
    'max_depth': [2, 3, 4],
    'subsample': [0.7, 0.8, 0.9],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [4, 5, 6] 
}

gbr = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator = gbr, scoring = 'neg_root_mean_squared_error',
                           param_grid = param_grid, cv = 3)

grid_search.fit(X_train, y_train)

grid_cv_results_df = pd.DataFrame(grid_search.cv_results_)
columns = ['param_n_estimators', 'param_learning_rate', 'param_subsample',
           'param_max_depth', 'param_min_samples_split', 'param_min_samples_leaf', 
           'mean_test_score','std_test_score', 'rank_test_score']

grid_cv_results_df[columns].sort_values("rank_test_score").head()

In [27]:
n_estimators = grid_search.best_params_["n_estimators"]
learning_rate = grid_search.best_params_["learning_rate"]
subsample = grid_search.best_params_["subsample"]
max_depth = grid_search.best_params_["max_depth"]
min_samples_split = grid_search.best_params_["min_samples_split"] 
min_samples_leaf = grid_search.best_params_["min_samples_leaf"]


best_gbr = GradientBoostingRegressor(n_estimators = n_estimators, learning_rate = learning_rate, 
                                     subsample = subsample, max_depth = max_depth, 
                                     min_samples_split = min_samples_split, 
                                     min_samples_leaf = min_samples_leaf, random_state = 42)


best_gbr.fit(X_train, y_train)

print('R-Squared score on training data : {}'.format(best_gbr.score(X_train, y_train)))
print('R-Squared score on test data     : {}'.format(best_gbr.score(X_test, y_test)))

# **Testing**

In this section, the performances of machine learning models are tested in detail. While performing this test, MAE, MSE, RMSE, R2 metrics were used. Looking at the metrics, it is seen that the two models perform close to each other. The point to note here is the difference between MAE and MSE metrics. Mean absolute error (MAE) measures the absolute mean distance between actual data and predicted data and treats all errors as equal, while mean squared error (MSE) penalizes large errors more. This shows that our models are very inaccurate in some predictions.

In [28]:
y_pred_test = best_rf.predict(X_test)
y_true = y_test
y_pred = y_pred_test

mae_rf = MAE(y_true, y_pred)
mse_rf = MSE(y_true, y_pred)
rmse_rf = MSE(y_true, y_pred, squared=False)
r2_rf = r2_score(y_true, y_pred)

y_pred_test = best_gbr.predict(X_test)
y_true = y_test
y_pred = y_pred_test

mae_gbr = MAE(y_true, y_pred)
mse_gbr = MSE(y_true, y_pred)
rmse_gbr = MSE(y_true, y_pred, squared=False)
r2_gbr = r2_score(y_true, y_pred)

data ={'Random Forest Regressor': [mae_rf, mse_rf, rmse_rf, r2_rf],
       'Gradient Boosting Regressor':[mae_gbr, mse_gbr, rmse_gbr, r2_gbr]}
  
df = pd.DataFrame.from_dict(data, orient ='index', columns=["MAE", "MSE", "RMSE", "R-Squared"]) 
df.index.names = ['Model']
df

Since the performances of the models are very close to each other, only the Random Forest Regressor model is used in the following examinations. First of all, feature importances for Random Forest Regressor are examined. In the graph below, it is seen that the features that have the most impact on the estimation of the medical cost are smoker,  bmi, and age features, respectively.

In [29]:
data ={'Feature':X.columns,
       'Importance': best_rf.feature_importances_}
  
importances_rf = pd.DataFrame.from_dict(data) 
importances_rf.sort_values("Importance", ascending=False, inplace=True)


fig = px.bar(importances_rf, x='Feature', y='Importance', text_auto='.4f')
fig.update_layout(title_text='Random Forest Feature Importance', 
                  title_x=0.5, title_y=0.95)
fig.show()

In this section, it is examined whether there is a pattern between erroneous predictions. Target and prediction values of the training and test sets are visualized in the scatter plots below. The 45-degree line is drawn to make it easier to see the difference between the target and prediction values. 

When we look at the non-smoker's graphs of the train and test sets, it is seen that many targets are relatively overestimated with small margins, and a smaller group is underestimated with large margins. The situation is slightly different in the graphs of smokers. Here, the target values are sometimes underestimated and sometimes overestimated by relatively small differences. But a very small group is underestimated by large margins. 

These errors can have multiple causes. First of all, of course, models are open to development. But it looks like the main problem here is the limited data. In some cases, when smoking or bmi features are ineffective, a feature that we do not have increases the medical costs. Therefore, the model cannot identify why patients with similar characteristics sometimes have more medical costs. Thus, the model tries to make the predictions somewhat between patients with high medical costs and patients with low medical costs. This causes the model to overestimate the medical costs of a large group by small margins while underestimating the medical costs of a small group by large margins.

In [30]:
rf_y_hat = best_rf.predict(X_train)
model_results = pd.DataFrame(y_train).reset_index().copy()
model_results.set_index('index', inplace = True)

model_results.rename(columns={"charges": "Target"}, inplace = True)
model_results.index.names = ['']

model_results["Prediction"] = rf_y_hat
model_results["Residual"] = model_results["Target"] - model_results["Prediction"]
model_results["Percentage Difference"] = np.absolute(model_results["Residual"] / model_results["Target"] * 100)

model_analysis = pd.merge(df_raw, model_results, left_index=True, right_index=True) 


fig = px.scatter(
    model_analysis.sort_values(["smoker","bmi"], ascending=True),
    x="Target", y="Prediction", 
    facet_col="smoker", color="bmi_bins", 
    color_discrete_sequence=["orange", "green","blue","red"]
)

fig.add_shape(
    type="line", 
    x0=0, x1=70000, y0=0, y1=70000, 
    line=dict(
        color="orange", 
        width=3
    ),row="all", col="all"
)

fig.update_layout(
    legend_title = 'BMI',
    title_text='',
    title_x=0.5, title_y=0.95
)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [31]:
rf_y_hat = best_rf.predict(X_test)
model_results = pd.DataFrame(y_test).reset_index().copy()
model_results.set_index('index', inplace = True)

model_results.rename(columns={"charges": "Target"}, inplace = True)
model_results.index.names = ['']

model_results["Prediction"] = rf_y_hat
model_results["Residual"] = model_results["Target"] - model_results["Prediction"]
model_results["Percentage Difference"] = np.absolute(model_results["Residual"] / model_results["Target"] * 100)

model_analysis = pd.merge(df_raw, model_results, left_index=True, right_index=True) 


fig = px.scatter(
    model_analysis.sort_values(["smoker","bmi"], ascending=True), 
    x="Target", y="Prediction", 
    facet_col="smoker", color="bmi_bins",
    color_discrete_sequence=["orange", "green","blue","red"]
)

fig.add_shape(
    type="line", 
    x0=0, x1=70000, y0=0, y1=70000, 
    line=dict(
        color="orange", 
        width=3
    ),row="all", col="all"
)

fig.update_layout(
    legend_title = 'BMI',
    title_text='', 
    title_x=0.5, title_y=0.95
)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()